# Development notebook for GAN architecture

I'll use this notebook to test out modifications to the DCGAN architecture.

In [35]:
# System
from __future__ import print_function
from __future__ import absolute_import
from __future__ import division

# Externals
import torch
import torch.nn as nn
from torch.autograd import Variable

## Testing out different conv params

Can I tweak the kernel sizes, strides, number of layers, and maintain a consistent input and output size?

In [36]:
n_samples = 1

In [37]:
# Create a sample image and noise for input
img_size = 64
x = Variable(torch.randn(n_samples, 1, img_size, img_size))
x.size()

torch.Size([1, 1, 64, 64])

In [38]:
noise_dim = 64
noise = Variable(torch.randn(n_samples, noise_dim, 1, 1))
noise.size()

torch.Size([1, 64, 1, 1])

### Default layer configuration

In [39]:
# Discriminator layer sizes (dropping BN and activations)
ndf = 16
print('input:', x.size())
h = nn.Conv2d(1, ndf, 4, 2, 1)(x)
print('conv1:', h.size())
h = nn.Conv2d(ndf, ndf*2, 4, 2, 1)(h)
print('conv2:', h.size())
h = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)(h)
print('conv3:', h.size())
h = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1)(h)
print('conv4:', h.size())
d = nn.Conv2d(ndf*8, 1, 4, 1, 0)(h)
print('conv5:', d.size())

input: torch.Size([1, 1, 64, 64])
conv1: torch.Size([1, 16, 32, 32])
conv2: torch.Size([1, 32, 16, 16])
conv3: torch.Size([1, 64, 8, 8])
conv4: torch.Size([1, 128, 4, 4])
conv5: torch.Size([1, 1, 1, 1])


In [40]:
# Generator layer sizes
ngf = 16
print('input:  ', noise.size())
h = nn.ConvTranspose2d(noise_dim, ngf*8, 4, 1, 0)(noise)
print('deconv1:', h.size())
h = nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1)(h)
print('deconv2:', h.size())
h = nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1)(h)
print('deconv3:', h.size())
h = nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1)(h)
print('deconv4:', h.size())
g = nn.ConvTranspose2d(ngf, 1, 4, 2, 1)(h)
print('deconv5:', g.size())

input:   torch.Size([1, 64, 1, 1])
deconv1: torch.Size([1, 128, 4, 4])
deconv2: torch.Size([1, 64, 8, 8])
deconv3: torch.Size([1, 32, 16, 16])
deconv4: torch.Size([1, 16, 32, 32])
deconv5: torch.Size([1, 1, 64, 64])


### With one extra layer in each

In [41]:
# Discriminator layer sizes (dropping BN and activations)
ndf = 16
print('input:', x.size())
h = nn.Conv2d(1, ndf, 4, 2, 1)(x)
print('conv1:', h.size())
h = nn.Conv2d(ndf, ndf*2, 4, 2, 1)(h)
print('conv2:', h.size())
h = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)(h)
print('conv3:', h.size())
h = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1)(h)
print('conv4:', h.size())
h = nn.Conv2d(ndf*8, ndf*16, 4, 2, 1)(h)
print('conv5:', h.size())
d = nn.Conv2d(ndf*16, 1, 2, 1, 0)(h)
print('conv6:', d.size())

input: torch.Size([1, 1, 64, 64])
conv1: torch.Size([1, 16, 32, 32])
conv2: torch.Size([1, 32, 16, 16])
conv3: torch.Size([1, 64, 8, 8])
conv4: torch.Size([1, 128, 4, 4])
conv5: torch.Size([1, 256, 2, 2])
conv6: torch.Size([1, 1, 1, 1])


In [42]:
# Generator layer sizes
ngf = 16
print('input:  ', noise.size())
h = nn.ConvTranspose2d(noise_dim, ngf*16, 2, 1, 0)(noise)
print('deconv1:', h.size())
h = nn.ConvTranspose2d(ngf*16, ngf*8, 4, 2, 1)(h)
print('deconv2:', h.size())
h = nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1)(h)
print('deconv3:', h.size())
h = nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1)(h)
print('deconv4:', h.size())
h = nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1)(h)
print('deconv5:', h.size())
g = nn.ConvTranspose2d(ngf, 1, 4, 2, 1)(h)
print('deconv6:', g.size())

input:   torch.Size([1, 64, 1, 1])
deconv1: torch.Size([1, 256, 2, 2])
deconv2: torch.Size([1, 128, 4, 4])
deconv3: torch.Size([1, 64, 8, 8])
deconv4: torch.Size([1, 32, 16, 16])
deconv5: torch.Size([1, 16, 32, 32])
deconv6: torch.Size([1, 1, 64, 64])


### Conditional GAN architecture

In this case I will have 1-2 parameters on which to condition the generator and discriminator.

In [43]:
cond_dim = 2
cond = Variable(torch.randn(n_samples, cond_dim))
cond.size()

torch.Size([1, 2])

In [45]:
# Discriminator layer sizes (dropping BN and activations)
ndf = 16
print('input:', x.size())
h = nn.Conv2d(1, ndf, 4, 2, 1)(x)
print('conv1:', h.size())
h = nn.Conv2d(ndf, ndf*2, 4, 2, 1)(h)
print('conv2:', h.size())
h = nn.Conv2d(ndf*2, ndf*4, 4, 2, 1)(h)
print('conv3:', h.size())
h = nn.Conv2d(ndf*4, ndf*8, 4, 2, 1)(h)
print('conv4:', h.size())

# This next bit is effectively just a FC layer.
# Let's drop the pretense to make it easier to work with.
#d = nn.Conv2d(ndf*8, 1, 4, 1, 0)(h)
h = h.view(n_samples, -1)
print('flat: ', h.size())
h = torch.cat([h, cond], dim=1)
print('concat:', h.size())

d = nn.Linear(h.size(1), 1)(h)
print('linear:', d.size())

input: torch.Size([1, 1, 64, 64])
conv1: torch.Size([1, 16, 32, 32])
conv2: torch.Size([1, 32, 16, 16])
conv3: torch.Size([1, 64, 8, 8])
conv4: torch.Size([1, 128, 4, 4])
flat:  torch.Size([1, 2048])
concat: torch.Size([1, 2050])
linear: torch.Size([1, 1])


In [48]:
# Concatenate the noise and condition variables
noise_cond = torch.cat([noise, cond[:, :, None, None]], dim=1)

# Generator layer sizes
ngf = 16
print('input:  ', noise_cond.size())
h = nn.ConvTranspose2d(noise_dim + cond_dim, ngf*8, 4, 1, 0)(noise_cond)
print('deconv1:', h.size())
h = nn.ConvTranspose2d(ngf*8, ngf*4, 4, 2, 1)(h)
print('deconv2:', h.size())
h = nn.ConvTranspose2d(ngf*4, ngf*2, 4, 2, 1)(h)
print('deconv3:', h.size())
h = nn.ConvTranspose2d(ngf*2, ngf, 4, 2, 1)(h)
print('deconv4:', h.size())
g = nn.ConvTranspose2d(ngf, 1, 4, 2, 1)(h)
print('deconv5:', g.size())

input:   torch.Size([1, 66, 1, 1])
deconv1: torch.Size([1, 128, 4, 4])
deconv2: torch.Size([1, 64, 8, 8])
deconv3: torch.Size([1, 32, 16, 16])
deconv4: torch.Size([1, 16, 32, 32])
deconv5: torch.Size([1, 1, 64, 64])
